In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install scikit-learn==0.24 --user

     |████████████████████████████████| 22.3 MB 613 kB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyldavis 3.3.1 requires numpy>=1.20.0, but you have numpy 1.19.5 which is incompatible.
pdpbox 0.2.1 requires matplotlib==3.1.1, but you have matplotlib 3.4.2 which is incompatible.
hypertools 0.7.0 requires scikit-learn!=0.22,<0.24,>=0.19.1, but you have scikit-learn 0.24.0 which is incompatible.


In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('https://raw.githubusercontent.com/plenoi/Clinic/master/ultima_all_clean.csv')
df = df.set_index('hn') # column แรกเป็น hn
df.head() # จะมองเห็นแค่ 5 บรรทัดแรกเท่านั้น

,age,parity,hiv,menopaus,disease,surgery,conization,OPDsize,appearance,stage,...,nodeyiel,RHlvsi,depth,size,utmet,vgmargin,vgmet,pelvicme,pmmet,adnmet
hn,,,,,,,,,,,,,,,,,,,,,
2631840,52,3,0.0,0.0,1,1,0.0,5.0,4.0,5.0,...,21.0,0.0,3.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0
2633481,32,2,0.0,0.0,0,1,0.0,5.0,1.0,5.0,...,11.0,0.0,3.0,5.0,0.0,0.0,0.0,1.0,0.0,2.0
2634477,52,2,0.0,0.0,0,0,0.0,5.0,NaN,5.0,...,35.0,6.0,3.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0
2633633,38,2,0.0,0.0,0,0,0.0,2.0,1.0,4.0,...,20.0,16.0,3.0,3.8,0.0,0.0,0.0,0.0,0.0,2.0
2630496,55,3,0.0,1.0,0,0,1.0,0.0,5.0,4.0,...,17.0,9.0,3.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [5]:
df.shape 

(1723, 23)

1723 row , 23 col

In [6]:
df.isnull().sum(0) #รวมค่า จากแกรน x

age             0
parity          0
hiv             4
menopaus        1
disease         0
surgery         0
conization      5
OPDsize        17
appearance    101
stage          24
pchemo          1
Wardsize      145
finalhisto     10
nodeyiel       12
RHlvsi        366
depth         489
size          114
utmet          98
vgmargin       96
vgmet          97
pelvicme        1
pmmet          94
adnmet          7
dtype: int64

In [7]:
df_clean_column = df.drop(['appearance','Wardsize','RHlvsi','depth','nodeyiel','vgmargin','pelvicme','adnmet'], 1)
df_clean_column.isnull().sum(0)

age             0
parity          0
hiv             4
menopaus        1
disease         0
surgery         0
conization      5
OPDsize        17
stage          24
pchemo          1
finalhisto     10
size          114
utmet          98
vgmet          97
pmmet          94
dtype: int64

In [8]:
df_clean = df_clean_column.dropna(0) # drop every line if that have missing value

In [9]:
df_clean.shape # draft มาจาก 

(1555, 15)

In [10]:
pel_class = np.unique(df_clean['pmmet'])
pel_class

array([0., 1.])

In [11]:
pel_value = [sum(df_clean['pmmet']==pel_class[0]),
             sum(df_clean['pmmet']==pel_class[1])]
pel_value

[1278, 277]

In [12]:
y = df_clean['pmmet'].values
X = df_clean.drop(['pmmet'],1).values #ทำการ drop pmmet ออกไปจาก df_clean ทั้ง column

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [14]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1)) # min max ของ scaler
scaler.fit(X_train)
X_train_norm = scaler.transform(X_train)
X_test_norm = scaler.transform(X_test)

# WEEK 7

# Hold Out 

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.2,random_state=0) #test ห้ามยุ่ง

In [16]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1)) # min max ของ scaler
scaler.fit(X_train)
X_train_norm = scaler.transform(X_train)
X_val_norm = scaler.transform(X_val)
#เป็นการปรับข้อมูลให้อยู่ในช่วย 0 กับ 1

In [17]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, solver='liblinear') # model name
clf.fit(X_train_norm, y_train)
yp = clf.predict(X_val_norm)
acc = sum (yp == y_val) / len(y_val)
print("Hold Out Acc. : "+str(acc*100))

Hold Out Acc. : 84.56591639871382


# Cross 

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.2,random_state=0)

In [19]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1)) # min max ของ scaler
scaler.fit(X_train)
X_train_norm = scaler.transform(X_train)

In [20]:
from sklearn.model_selection import cross_val_score 
acc = cross_val_score(clf,X_train_norm, y_train,cv = 10)
print("10CV Trainning Accuracy : " +str(acc.mean()*100) ) # ใช้ mean เพราะ acc cross มีหลายค่าซึ่งจำเป็นต้องหาค่าเฉลี่ย

10CV Trainning Accuracy : 83.36193548387098


# PCA 

In [21]:
# PCA มักทำก่อน norm
from sklearn.decomposition import PCA
pca = PCA(n_components = 2)
pca.fit(X_train)
X_train_pca = pca.transform(X_train)
X_test_pca =  pca.transform(X_test)

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1)) # min max ของ scaler
scaler.fit(X_train_pca)
X_train_norm = scaler.transform(X_train_pca)
X_test_norm = scaler.transform(X_test_pca)

In [22]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, solver='liblinear') # model name
clf.fit(X_train_norm, y_train)
yp = clf.predict(X_test_norm)
acc = sum (yp == y_test) / len(y_test)
print("Test Acc. : "+str(acc * 100))

Test Acc. : 83.92282958199357


# SelectKBest feature_selection

In [23]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
fs = SelectKBest(f_classif, k=5)
fs.fit(X_train,y_train)
X_train_fs = fs.transform(X_train)
X_test_fs = fs.transform(X_test)

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1)) # min max ของ scaler
scaler.fit(X_train_fs)
X_train_norm = scaler.transform(X_train_fs)
X_test_norm = scaler.transform(X_test_fs)

In [24]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, solver='liblinear') # model name
clf.fit(X_train_norm, y_train)
yp = clf.predict(X_test_norm)
acc = sum (yp == y_test) / len(y_test)
print("Test Acc. : "+str(acc * 100))

Test Acc. : 85.53054662379421


# SFS

In [25]:
#from sklearn.feature_selection import SequentialFeatureSelector
#clf = LogisticRegression(random_state=0, solver='liblinear')
#sfs = SequentialFeatureSelector(clf, n_features_to_select=3)
#sfs.fit(X_train, y_train)
#X_train_sfs = sfs.transform(X_train)
#X_test_sfs = sfs.transform(X_test)

# Pipeline (Assignment)

In [26]:
df_clean.shape

(1555, 15)

In [27]:
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

sm = SMOTE(random_state=1)
X_res, y_res = sm.fit_resample(X_train, y_train)
params = params = {'C': [1, 2, 4, 8, 16,32]}

clf = Pipeline(
    [
        #('feature_reduction',PCA(n_components = 14)), #n_com = 14 is max acc.
        ('standardScaler',StandardScaler()),
        ('scalar',MinMaxScaler(feature_range=(0,1))),
        ('feature_selection',SelectKBest(f_classif,k=10)), # k = 10 is max acc.
        ('feature_fwd', SequentialFeatureSelector(LogisticRegression(random_state=0, solver='liblinear') , n_features_to_select=9)), # n_f select = 9 is max acc.
        ('classification',GridSearchCV(LogisticRegression(random_state=0, solver='liblinear'),params, cv=5)) # change cv not change acc. for this 
    ]
)

clf.fit(X_res,y_res)

from sklearn.model_selection import cross_val_score 
acc = cross_val_score(clf,X_res ,y_res,cv = 10) # cv = 10 is max acc.
print("Trainning Accuracy : " +str(acc.mean()*100))

Trainning Accuracy : 77.40196078431372


**#Model train result with balance data#**
1. Logistic regression = 76.3235294117647 with full Pipeline  // 77.40196078431372 with remove PCA  // 76.02941176470588 with remove PCA,Kbest // 77.20588235294117 remove PCA,Scalar

**#Model train result with inbalance data**
1. Logistic_regression = 82.88064516129032  // 83.36661546338965  // 83.44049398162629 // 83.44512925496005 at cv = 5 and acc cv = 17 
2. GaussianNB = 81.99577498529955 at cv 5-10 and acc cv 15-17
3. DecisionTreeClassifier = 81.91483870967741 at cv 5 and acc cv 10

In [28]:
#test result
yp = clf.predict(X_test)
acc = sum(yp == y_test)/len(y_test)
print("Test Training Acc. : " + str(acc*100))

Test Training Acc. : 72.66881028938906


# **Over fit : True**

 # END WEEK7

In [29]:
df_7 = df.drop_duplicates()
df_7.shape

(1723, 23)

In [30]:
s = df.corr().unstack()
so = s.sort_values(ascending=False)
so

age         age           1.000000
finalhisto  finalhisto    1.000000
menopaus    menopaus      1.000000
disease     disease       1.000000
surgery     surgery       1.000000
                            ...   
conization  Wardsize     -0.711525
appearance  Wardsize     -0.757519
Wardsize    appearance   -0.757519
appearance  OPDsize      -0.790494
OPDsize     appearance   -0.790494
Length: 529, dtype: float64

In [31]:
import sklearn
sklearn.__version__

'0.24.0'